# Notebook

In [78]:
import math
import yaml

## Book Keeping

In [79]:
MODEL = "PE1R_112_M4"
VERBOSITY = 1

In [80]:
SWITCHING_FREQUENCY = 5000
BUS_VOLTAGE = 760
PEAK_CARRIER_VOLTAGE = 2
GEARBOX_GAIN = 10
LOAD_MOMENT_OF_INERTIA = 8.1
LOAD_FRICTION_COEFFICIENT = 4.5
CURRENT_SENSOR_GAIN = 1 / 30
CURRENT_RESPONSE_TIME = 20e-3
CURRENT_OVERSHOOT = 7
SPEED_SENSOR_GAIN =  2 / 100
SPEED_RESPONSE_TIME = 200e-3
SPEED_OVERSHOOT = 5
POSITION_SENSOR_GAIN =  1 / 330
POSITION_RESPONSE_TIME = 2000e-3
POSITION_OVERSHOOT = 4

In [81]:
with open("../reports/" + MODEL + "_parameters.yaml", "r") as file:
    parameters = yaml.safe_load(file)

In [82]:
Kc = 2 * BUS_VOLTAGE / PEAK_CARRIER_VOLTAGE
Ts = 1 / SWITCHING_FREQUENCY

if VERBOSITY:
    print("Kc :: {0}".format(Kc))

Kc :: 760.0


In [83]:
Rs = parameters["provided_parameters"]["Rs"]
Ld = parameters["estimated_parameters"]["Ld"]

tau_maid = Ld / Rs
Kmaid = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfid = (CURRENT_RESPONSE_TIME / 10) / 3 # 10x faster than the actual current loop
zeta_id = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpid = ((2 * tau_maid) / tau_mfid - 1) / Kmaid
Tiid = (tau_mfid**2 * zeta_id**2 * Kmaid) / tau_maid
tau_zid = Kpid * Tiid

if VERBOSITY:
    print("Kpid :: {0}".format(Kpid))
    print("Tiid :: {0}".format(Tiid))
    print("tau_zid :: {0}".format(tau_zid))

Kpid :: 2.1256578947368423
Tiid :: 0.00025752692561198815
tau_zid :: 0.0005474141425344301


In [84]:
Lq = parameters["estimated_parameters"]["Lq"]

tau_maiq = Lq / Rs
Kmaiq = (Kc * CURRENT_SENSOR_GAIN) / Rs
tau_mfiq = CURRENT_RESPONSE_TIME / 3
zeta_iq = -math.log(CURRENT_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(CURRENT_OVERSHOOT / 100)**2)
Kpiq = ((2 * tau_maiq) / tau_mfiq - 1) / Kmaiq
Tiiq = (tau_mfiq**2 * zeta_iq**2 * Kmaiq) / tau_maiq
tau_ziq = Kpiq * Tiiq

if VERBOSITY:
    print("Kpiq :: {0}".format(Kpiq))
    print("Tiiq :: {0}".format(Tiiq))
    print("tau_ziq :: {0}".format(tau_ziq))

Kpiq :: 0.18059210526315786
Tiiq :: 0.02575269256119882
tau_ziq :: 0.00465073296582176


In [85]:
Br = parameters["estimated_parameters"]["Br"]
Jr = parameters["provided_parameters"]["Jr"]
Ke = parameters["estimated_parameters"]["Ke"]
Kt = parameters["estimated_parameters"]["Kt"]

Jeq = Jr + LOAD_MOMENT_OF_INERTIA / GEARBOX_GAIN**2
Beq = Br + LOAD_FRICTION_COEFFICIENT / GEARBOX_GAIN**2
Jti = (Ke * Kt * Tiiq) / (Kc * CURRENT_SENSOR_GAIN)
Ksv = SPEED_SENSOR_GAIN * (30 / math.pi) 

tau_mav = (Jeq + Jti) / Beq
Kmav = (Kt * Ksv) / (Beq * GEARBOX_GAIN * CURRENT_SENSOR_GAIN)
tau_mfv = SPEED_RESPONSE_TIME / 3
zeta_v = -math.log(SPEED_OVERSHOOT / 100) / math.sqrt(math.pi**2 + math.log(SPEED_OVERSHOOT / 100)**2)
Kpv = ((2 * tau_mav) / tau_mfv - 1) / Kmav
Tiv = (tau_mfv**2 * zeta_v**2 * Kmav) / tau_mav
tau_zv = Kpv * Tiv

if VERBOSITY:
    print("Kpv :: {0}".format(Kpv))
    print("Tiv :: {0}".format(Tiv))
    print("tau_zv :: {0}".format(tau_zv))


Kpv :: 2.0201086838649522
Tiv :: 0.0308562022066056
tau_zv :: 0.062332882028656875


In [86]:
Ksp = POSITION_SENSOR_GAIN * (180 / math.pi)
Kmap = Ksp / Ksv
tau_mfp = POSITION_RESPONSE_TIME / 3
Kpv = 1 / (Kmap * tau_mfp)

if VERBOSITY:
    print("Kpv :: {0}".format(Kpv))

Kpv :: 1.6500000000000004
